In [1]:
import threading

def func(id):
    print("Thread: ",id," running\n")

if __name__=="__main__":
    t1 = threading.Thread(target=func, args=(1,))
    t2 = threading.Thread(target=func, args=(2,))

    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("Thread terminati")

    threads = list()

    for i in range(1,5):
        t=threading.Thread(target=func, args=("T"+str(i),))
        threads.append(t)
        t.start()
    
    for index, t in enumerate(threads):
        print("Joining thread: ",index)
        t.join()

Thread:  1  running

Thread:  2  running

Thread terminati
Thread:  T1  running

Thread:  T2  running

Thread:  T3  running

Thread:  T4  running

Joining thread:  0
Joining thread:  1
Joining thread:  2
Joining thread:  3


In [2]:
import threading
import logging
import random

logging.basicConfig(level=logging.DEBUG, format='(%(threadName)-0s)%(message)s',)

def show(d):
    try:
        val = d.val
    except:
        logging.debug('No value yet!')
    else:
        logging.debug('value=%s',val)

def f(d):
    show(d)
    d.val=random.randint(1,100)
    show(d)

if __name__=="__main__":
    d = threading.local()
    show(d)
    d.val = 999
    show(d)

    for i in range(2):
        t = threading.Thread(target=f, args=(d,))
        t.start()

(MainThread)No value yet!
(MainThread)value=999
(Thread-11 (f))No value yet!
(Thread-12 (f))No value yet!
(Thread-11 (f))value=50
(Thread-12 (f))value=63


In [4]:
# Utilizzo classe Lock

import threading
from threading import Lock
import time

x = 10

def increment(increment_by, lock):
    global x
    lock.acquire()
    local_counter = x
    local_counter += increment_by

    time.sleep(1)

    x=local_counter

    print(f'{threading.current_thread().name} increments x by {increment_by}, x:{x}')
    lock.release()

lock = Lock()

t1 = threading.Thread(target=increment, args=(5,lock))
t2 = threading.Thread(target=increment, args=(10,lock))

t1.start()
t2.start()

t1.join()
t2.join()

print(f'The final value of x is {x}')

Thread-15 (increment) increments x by 5, x:15
Thread-16 (increment) increments x by 10, x:25
The final value of x is 25


In [6]:
# Utilizzo di Semaphore

from threading import *
from time import sleep
from random import random


obj = Semaphore(3)

def display(name):
    obj.acquire()
    value = random()
    sleep(value)
    print(f'Thread {name} got {value}')
    obj.release()

if __name__=="__main__":
    threads = []

    for i in range(10):
        t = Thread(target=display, args=('Thread-'+str(i),))
        threads.append(t)
        t.start()
    for thread in threads:
        print(thread)
        thread.join()

<Thread(Thread-17 (display), started 16692)>
Thread Thread-0 got 0.23463585816900945
<Thread(Thread-18 (display), started 19312)>
Thread Thread-2 got 0.5760702074290214
Thread Thread-1 got 0.612820722073765
<Thread(Thread-19 (display), stopped 2280)>
<Thread(Thread-20 (display), started 12932)>
Thread Thread-3 got 0.37552860476363925
<Thread(Thread-21 (display), started 13440)>
Thread Thread-4 got 0.15530545716680344
<Thread(Thread-22 (display), started 7092)>
Thread Thread-7 got 0.024493414561535976
Thread Thread-6 got 0.6863320591217511
Thread Thread-5 got 0.8361177730346977
<Thread(Thread-23 (display), stopped 7648)>
<Thread(Thread-24 (display), stopped 5112)>
<Thread(Thread-25 (display), started 8220)>
Thread Thread-8 got 0.6893062897596386
<Thread(Thread-26 (display), started 10504)>
Thread Thread-9 got 0.7113376654158448


In [7]:
# utilizzo della classe Event

import threading as thd

import time

def task(event, timeout):
    print("Started thread but waiting for event...")
    event_set = event.wait(timeout)
    if event_set:
        print("Event received, releasing thread...")
    else:
        print("Time out, moving ahead...")

if __name__=="__main__":
    e = thd.Event()

    thread = thd.Thread(name='Event-Thread',target=task,args=(e,4))
    thread.start()

    time.sleep(3)

    e.set()
    print("Event is set.")

    thread.join()
    print("Event-Thread joined")

Started thread but waiting for event...
Event is set.Event received, releasing thread...

Event-Thread joined


In [1]:
import multiprocess as mp

def func():
   print ('Process running')
   return


if __name__ == '__main__':

    # creating process
    p = mp.Process(target = func)

    # starting process
    p.start()

    # wait until the process finishes
    p.join()

    print("all joined")

all joined


In [1]:
import multiprocess as mp

class MyProcess(mp.Process):
    def run(self):
        print("Process running")
        return
if __name__ == "__main__":
    p = MyProcess()

    p.start()

    p.join()

In [1]:
# ProdCons con Semafori

import logging
import threading
import time
from random import randint

CONSUMER = 'Consumer'
PRODUCER = 'Producer'
N_CONSUMERS = 10
N_PRODUCERS = 10
QUEUE_SIZE = 5

logging.basicConfig(level=logging.DEBUG, format='[%(threadName)-0s] %(message)s',)

def get(queue):
    return queue.pop(0)

def set(queue):
    item = randint(0,100)
    queue.append(item)
    return item

class consumerThread(threading.Thread):
    def __init__(self, mutex, empty, full, queue, name):
        threading.Thread.__init__(self, name=name)
        self.mutex=mutex
        self.empty=empty
        self.full=full
        self.queue=queue
    def run(self):
        logging.debug('\t\t\tStarted')
        logging.debug('\t\t\tChecking full semaphore...')

        self.full.acquire()
        with self.mutex:
            logging.debug('\t\t\tAcquired mutex')
            time.sleep(1)
            item = get(self.queue)
            logging.debug('\t\t\tItem: %r', item)
            logging.debug('\t\t\tRelease mutex')
        self.empty.release()
        logging.debug('\t\t\tReleased empty semaphore')

def produce(mutex, empty, full, queue):
    logging.debug('\t\t\tStarted')
    logging.debug('\t\t\tChecking empty semaphore...')

    empty.acquire()
    with mutex:
        logging.debug('\t\t\tAcquired mutex')
        time.sleep(1)
        item = set(queue)
        logging.debug('\t\t\tItem: %r', item)
        logging.debug('\t\t\tRelease mutex')
    full.release()
    logging.debug('\t\t\tReleased full semaphore')

def main():
    queue = []

    mutex = threading.Semaphore()
    empty = threading.Semaphore(QUEUE_SIZE)
    full = threading.Semaphore(0)

    consumers = []
    producers = []

    for i in range (N_CONSUMERS):
        name = CONSUMER+str(i)
        ct = consumerThread(mutex,empty,full,queue,name)
        ct.start()
        consumers.append(ct)
    for i in range (N_PRODUCERS):
        pt = threading.Thread(target=produce, name=PRODUCER+str(i), args=(mutex,empty,full,queue),)
        pt.start()
        producers.append(pt)

    for i in range(N_CONSUMERS):
        consumers[i].join()
    for i in range(N_PRODUCERS):
        producers[i].join()
if __name__=="__main__":
    main()

[Consumer0] 			Started
[Consumer1] 			Started
[Consumer0] 			Checking full semaphore...
[Consumer2] 			Started
[Consumer3] 			Started
[Consumer4] 			Started
[Consumer1] 			Checking full semaphore...
[Consumer5] 			Started
[Consumer6] 			Started
[Consumer7] 			Started
[Consumer8] 			Started
[Consumer9] 			Started
[Producer0] 			Started
[Consumer2] 			Checking full semaphore...
[Producer1] 			Started
[Producer2] 			Started
[Producer3] 			Started
[Consumer3] 			Checking full semaphore...
[Producer4] 			Started
[Producer5] 			Started
[Producer6] 			Started
[Producer7] 			Started
[Consumer4] 			Checking full semaphore...
[Producer8] 			Started
[Producer9] 			Started
[Consumer5] 			Checking full semaphore...
[Consumer6] 			Checking full semaphore...
[Consumer7] 			Checking full semaphore...
[Consumer8] 			Checking full semaphore...
[Consumer9] 			Checking full semaphore...
[Producer0] 			Checking empty semaphore...
[Producer1] 			Checking empty semaphore...
[Producer2] 			Checking empty sema